# This notebook generates the recurrence plot images from raw ecg signals

Import libraries

In [1]:
import wfdb
import os
import numpy as np
from utils import *
from scipy import signal
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

Useful constants

In [2]:
NB_S_PER_MIN = 6000
SAMPLING_FREQUENCY = 100
LOWER_BOUND_FILTER = 1
UPPER_BOUND_FILTER = 48
NB_S_PER_HALF_MIN = 3000

train_files = [
    "a01",
    "c01",
    "b01",
    "a02",
    "c02",
    "b02",
    "a03",
    "c03",
    "b03",
    "a04",
    "c04",
    "b04",
    "a05",
    "c05",
    "a06",
    "c06",
    "a07",
    "c07",
    "a08",
    "c08",
    "a09",
    "a10",
    "a11",
    "a12",
    "a13",
    "a14",
    "a15",
    "a16",
]


test_files = ["b05", "c09", "c10", "a17", "a18", "a19", "a20"]

# assign directory
directory = "../ECG_Data/apnea-ecg-database-1.0.0/"

out_directory_train = "./Recurrence_Plots/Train/"
out_directory_test = "./Recurrence_Plots/Test/"


This function generates a matrix which is the recurrence plot from signal s

In [3]:
def rec_plot(s, eps=0.07):
    N = len(s)
    rows_repeated = np.repeat(s.reshape(1, N), N, axis=0)
    columns_repeated = np.repeat(s.reshape(N, 1), N, axis=1)

    return np.heaviside(-(np.absolute(columns_repeated - rows_repeated) - eps), 1)


This function generates the recurrence plot images in format png. to intended directory with a csv file that maps filenames to annotation(apnea or not)

In [4]:
def create_rp_images(source_directory, files, out_directory, csv_path):
    """generates the RP images in format png. to intended directory with a csv file
    that maps filenames to annotation(apnea or not)
    Args:
        source_directory:          string
        files:                     filenames list
        out_directory:             string
        csv_path:                  csv path for annotations
    """
    #create the directories
    splitted_out_directory = out_directory.split('/')
    os.mkdir(splitted_out_directory[1])
    os.mkdir( splitted_out_directory[1] + '/' + splitted_out_directory[2] )
    spc_labels_file = open(out_directory + csv_path, "a")
    spc_labels_file.write("image,label")
    plt.ioff()
    for filename in files:
        i = 0
        path = source_directory + filename

        print(filename)
        record = wfdb.rdrecord(path)
        apnea = wfdb.rdann(path, "apn")
        ecg = record.p_signal
        ecg = ecg.reshape(len(ecg))
        ecg = bandpass_signal(ecg, LOWER_BOUND_FILTER, UPPER_BOUND_FILTER)
        # choose minimum length
        length = min(
            len(apnea.symbol) - 1, int((len(ecg) - NB_S_PER_HALF_MIN) / NB_S_PER_MIN)
        )

        for j in range(length):
            label = apnea.symbol[j + 1]
            segment = ecg[
                NB_S_PER_HALF_MIN
                + (NB_S_PER_MIN * j) : NB_S_PER_HALF_MIN
                + (NB_S_PER_MIN * (j + 1))
            ]

            # save recurrence plot
            rp_file_name = "rp_" + filename + "_" + str(i) + ".png"

            plt.matshow(rec_plot(segment), cmap="Greys")

            plt.axis("off")
            plt.savefig(
                out_directory + rp_file_name, bbox_inches="tight", pad_inches=0.0
            )
            plt.close()
            # add line to csv
            spc_labels_file.write(rp_file_name + "," + label + "\n")
            # increment 1
            i += 1

    spc_labels_file.close()

## We create the RP for train and test

In [5]:
create_rp_images(directory, train_files, out_directory_train, "rp_train_labels.csv")

a01


KeyboardInterrupt: 

In [ ]:
create_rp_images(directory, test_files, out_directory_test, "rp_test_labels.csv")